In [1]:
from math import gamma
from time import sleep

import torch

compute_device = torch.device('cuda' if torch.cuda.is_available() else 'mps')
cpu_device = torch.device('cpu')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
torch.cuda.is_available()

True

# Env

In [4]:
def make_env(render_mode="rgb_array"):
    env = gym.make("gym_pusht/PushT-v0", render_mode=render_mode)
    return env

In [5]:
import gymnasium as gym
import gym_pusht
from time import sleep

env = make_env()

In [ ]:
test_env = make_env(render_mode="human")

observation, info = test_env.reset()

for _ in range(50):
    sleep(0.2)
    action = test_env.action_space.sample()
    observation, reward, terminated, truncated, info = test_env.step(action)
    image = test_env.render()

    if terminated or truncated:
        observation, info = test_env.reset()

test_env.close()

In [ ]:
test_env.close()

Implement the deepq Learning to compete the task

# Part 1: Numerical PushT state

## Observation Space

If obs_type is set to state, the observation space is a 5-dimensional vector representing the state of the environment: [agent_x, agent_y, block_x, block_y, block_angle]. The values are in the range [0, 512] for the agent and block positions and [0, 2*pi] for the block angle.

If obs_type is set to environment_state_agent_pos the observation space is a dictionary with: - environment_state: 16-dimensional vector representing the keypoint locations of the T (in [x0, y0, x1, y1, ...] format). The values are in the range [0, 512]. - agent_pos: A 2-dimensional vector representing the position of the robot end-effector.

If obs_type is set to pixels, the observation space is a 96x96 RGB image of the environment.

In [ ]:
env.observation_space

The Policy model is designed to give the action $a$ given the state $s$.

input:

$$
[agent_x, agent_y, t_x, t_y, t_{angle}]
$$

output

$$
[move_x, move_y]
$$


so the model tells

$$
a = P(s)
$$

In [ ]:
# from torch import nn
# import torch.nn.functional as F
#
#
# class Policy(nn.Module):
#     def __init__(self):
#         super(Policy, self).__init__()
#         self.fc1 = nn.Linear(env.observation_space.shape[0], 64)
#         self.fc2 = nn.Linear(64, 64)
#         self.fc3 = nn.Linear(64, 2)
#
#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#
#         return x
# model = Policy()
# input = torch.rand((4, 5))
#
# input
# output = model(input)
#
# print(output.shape)
# output.detach().numpy()

In [ ]:
# class Critic(nn.Module):
#     def __init__(self):
#         super(Critic, self).__init__()
#         self.fc1 = nn.Linear(env.observation_space.shape[0] + env.action_space.shape[0], 64)
#         self.fc2 = nn.Linear(64, 64)
#         self.fc3 = nn.Linear(64, 1)
#
#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = F.relu(self.fc3(x))
#
#         return x
# model = Critic()
# input = torch.rand((4, 7))
#
# input
# output = model(input)
#
# print(output.shape)
# output.detach().numpy()

Critic model is aimed to estimate

$$
Q(s,a)
$$

it takes concat input $[s,a]$, more detailed

input

$$
[agent_x, agent_y, t_x, t_y, t_{angle}, move_x, move_y]
$$

output

$$
score
$$


so it give single estimate of $Q$ value

$$
score = Q(s,a)
$$

The Policy model is trained using Critic model

Since we cant directly access the action value $Q(s,a)$ we use the critic model that gives estimate $Q'(s,a)$ to understand the value of this state action pair. Then we could use this estimate to compute the loss


$$
loss = -Q'(s,a)
$$

\- is used because optimization tasks aim to minimize function, hence minimizing -f is equivalent to maximizing f

In [ ]:
# def train_policy(policy_model, critic_model, input, optimizer):
#     """
#
#     Args:
#         policy_model: Model to update
#         critic_model: Critic model to compute the value of the proposed action
#         input: the input of shape [batch_size, 5] = (batch_size, observation_space.shape)
#         optimizer: optimizer for the policy model
#
#     Returns:
#
#     """
#     optimizer.zero_grad()
#
#     input = input.to(compute_device)
#
#     output = policy_model(input)
#
#     critic_input = torch.cat((input, output), dim=1)
#
#     score = critic_model(critic_input)
#
#     loss = -score
#
#     loss.backward()
#     optimizer.step()
#
#     return loss.item()

For training critic, we use the actual reward we received from the env, bootstrap with critic model and train it with basic MSE loss

In [ ]:
# def train_critic(critic_model, input, target, optimizer):
#     """
#     Train function that run one update on the critic network using batch of inputs.
#
#     Args:
#         critic_model: Model to update
#         input: the input of shape [batch_size, 7] = (batch_size, observation_space.shape + action_space.shape)
#         target: Is the target reward we received from env and critic_estimate model
#         optimizer: optimizer for the critic model
#
#     Returns:
#
#     """
#     optimizer.zero_grad()
#     criterion = nn.MSELoss()
#
#     critic_model.train()
#
#     input = input.to(compute_device)
#
#     output = critic_model(input)
#
#     loss = criterion(output, target)
#
#     loss.backward()
#     optimizer.step()
#
#     return loss.item()

so in my approach the policy actually changes, but I could ignore the fact that actions were recorded under another distribution because I am using Q, which gives some level of abstraction which is stable for the environment and any optimal policy will converge to identical function. Then the agent here is just the another function that is trained on this level of the representation of the environment. If I had the training process without Q where the policy is responsible to somehow incorporate the knowledge of the environment inside itself then I need to also think of sampling, because on the interpretation level the actions were done on that perception of env, which changed with policy.

this is idea of the model-based and model-free and particaul;larty off-policy and on-policy in the fact that env is not encoded in the model

Here comes the problem since the update rule is based on MSE of the value
$$
(Q(s,a), R(s,a))
$$

where target $R(s,a)$ is calculated

$$
R(s,a) = r + \gamma \cdot Q(s', \pi(s'))
$$

then the update rule:

$$
Q(s,a) \leftarrow \alpha \cdot (Q(s,a) - (r + \gamma \cdot Q(s', \pi(s'))))^2
$$

To solve this problem, I use the target models, this is the same duplicate of the Critic Model that is training with delay from the actual model, this will ensure more stable training process and prevent explode, when update depends on itself.

In [6]:
import random


class Memory:
    def __init__(self, batch_size=128, max_size=1024, min_sample=4096):
        self.batch_size = batch_size
        self.max_size = max_size
        self.min_sample = min_sample

        self.states = []
        self.actions = []
        self.rewards = []
        self.next_states = []
        self.dones = []

    def add(self, state, action, reward, next_state, done):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.next_states.append(next_state)
        self.dones.append(done)

        if len(self.states) > self.max_size:
            self._pop_oldest()

    def _pop_oldest(self):
        self.states.pop(0)
        self.actions.pop(0)
        self.rewards.pop(0)
        self.next_states.pop(0)

    def enough_sample(self):
        return len(self.states) >= self.min_sample

    def sample(self):
        idxs = random.sample(range(len(self.states)), self.batch_size)
        batch = dict(
            states=torch.stack([self.states[i] for i in idxs]),
            actions=torch.stack([self.actions[i] for i in idxs]),
            rewards=torch.stack([self.rewards[i] for i in idxs]),
            next_states=torch.stack([self.next_states[i] for i in idxs]),
            dones=torch.stack([self.dones[i] for i in idxs])
        )
        return batch


In [18]:
import numpy as np
from AIA.rl.pushT.Nets import Policy, Critic, OUNoise
from AIA.rl.pushT.trainer import train_critic, train_policy, soft_update


def train(policy, critic, policy_target, critic_target, optimizer_policy, optimizer_critic,
          memory, episodes=4000, max_steps=1000, gamma=0.99, tau=0.005, living_cost=-0.001):
    env = make_env()

    # Initialize noise with linear decay over all episodes
    ou_noise = OUNoise(env.action_space.shape[0],
                      initial_sigma=25,
                      final_sigma=1,
                      decay_steps=episodes * 500)  # Decay over total episodes

    # Statistics tracking
    reward_buffer = []
    critic_loss_buffer = []
    policy_loss_buffer = []
    print_interval = 25
    stats_window = 100

    for episode in range(episodes):
        state, _ = env.reset()
        episode_reward = 0
        prev_reward = 0
        episode_critic_loss = []
        episode_policy_loss = []

        for t in range(max_steps):
            # Action selection and environment step
            with torch.no_grad():
                state_tensor = torch.tensor(state, dtype=torch.float).to(compute_device) / 512.0
                action = policy(state_tensor).cpu().numpy()

            # Apply noise with current sigma
            noise = ou_noise.sample()

            action = action + noise

            action_env = np.clip(action, 0, 512)

            # Environment interaction
            next_state, reward, terminated, truncated, _ = env.step(action_env)
            done = terminated or truncated

            # Reward calculation
            shaped_reward = 100 * (reward - prev_reward) + living_cost
            prev_reward = reward

            # Store transition
            memory.add(torch.tensor(state, dtype=torch.float)/512.0,
                      torch.tensor(action, dtype=torch.float),
                      torch.tensor(shaped_reward, dtype=torch.float),
                      torch.tensor(next_state, dtype=torch.float)/512.0,
                      torch.tensor(int(done)))

            episode_reward += shaped_reward
            state = next_state

            # Training step
            if memory.enough_sample():
                batch = memory.sample()
                states = batch['states'].to(compute_device)
                actions = batch['actions'].to(compute_device)
                rewards = batch['rewards'].to(compute_device)
                next_states = batch['next_states'].to(compute_device)
                dones = batch['dones'].to(compute_device)

                # Critic update
                with torch.no_grad():
                    target_actions = policy_target(next_states)
                    target_actions = torch.clamp(target_actions, 0, 512)
                    target_q = critic_target(torch.cat((next_states, target_actions), 1))
                    targets = rewards + gamma * (1 - dones) * target_q.squeeze(-1)  # Match shapes

                critic_loss = train_critic(critic, torch.cat((states, actions), 1), targets,
                                         optimizer_critic, compute_device)
                episode_critic_loss.append(critic_loss)

                # Policy update
                policy_loss = train_policy(policy, critic, states, optimizer_policy, compute_device)
                episode_policy_loss.append(policy_loss)

                # Target updates
                soft_update(policy_target, policy, tau)
                soft_update(critic_target, critic, tau)

            if done:
                break

        # Store episode statistics
        avg_critic_loss = np.mean(episode_critic_loss) if episode_critic_loss else 0
        avg_policy_loss = np.mean(episode_policy_loss) if episode_policy_loss else 0

        reward_buffer.append(episode_reward)
        critic_loss_buffer.append(avg_critic_loss)
        policy_loss_buffer.append(avg_policy_loss)

        # Print statistics every 100 episodes
        if (episode + 1) % print_interval == 0 or episode == episodes - 1:
            start_idx = max(0, len(reward_buffer) - stats_window)

            mean_reward = np.mean(reward_buffer[start_idx:])
            mean_critic = np.mean(critic_loss_buffer[start_idx:])
            mean_policy = np.mean(policy_loss_buffer[start_idx:])

            print(f"Episodes {episode+1}: "
                  f"Mean Reward: {mean_reward:.2f}, "
                  f"Critic Loss: {mean_critic:.4f}, "
                  f"Policy Loss: {mean_policy:.4f}, "
                  f"Noise Sigma: {ou_noise.current_sigma():.2f}")

    return reward_buffer



In [19]:
policy_model = Policy(env).to(compute_device)
critic_model = Critic(env).to(compute_device)
policy_target = Policy(env).to(compute_device)
critic_target = Critic(env).to(compute_device)
policy_target.load_state_dict(policy_model.state_dict())
critic_target.load_state_dict(critic_model.state_dict())
optimizer_policy = torch.optim.Adam(policy_model.parameters(), lr=1e-4)
optimizer_critic = torch.optim.Adam(critic_model.parameters(), lr=1e-3)
memory = Memory(batch_size=512, max_size=40000, min_sample=1024)

In [ ]:
train(
    policy_model,
    critic_model,
    policy_target,
    critic_target,
    optimizer_policy,
    optimizer_critic,
    memory,
    episodes= 10000,
    max_steps = 1000,
    gamma = 0.99,
    tau = 0.005)

In [21]:
test_env = make_env(render_mode="rgb_array")

state, _ = test_env.reset()

for _ in range(50):
    sleep(0.2)

    input = torch.tensor(state, dtype=torch.float).to(compute_device) / 512.0

    action = policy_model(input)

    new_state, reward, terminated, truncated, _ = test_env.step(action)

    state = new_state

    # image = test_env.render()

    if terminated or truncated:
        observation, info = test_env.reset()

test_env.close()

KeyboardInterrupt: 

In [ ]:
test_env.close()